In [7]:
import cantools
import can
from pprint import pprint 
db = cantools.database.load_file('C:/Users/U/Documents/4.Semester/Masterarbeit/concept_implementation/data/CAN_data/MassCAN_DBC_20231108_SmartForestry.dbc')
db.messages

[message('Engine_Std', 0x181, False, 7, None),
 message('Engine_Ext', 0x182, False, 8, None),
 message('DT_Std', 0x183, False, 8, None),
 message('DT_Ext', 0x184, False, 8, None),
 message('Cab_Std', 0x185, False, 8, None),
 message('Crane_Hyd', 0x186, False, 8, None)]

In [13]:
def process_log_file(log_file_path):
    with open(log_file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(' ')
            timestamp = parts[0]
            can_interface = parts[1]
            frame_id, data = parts[2].split('#')
            frame_id = int(frame_id, 16) 
            data = bytes.fromhex(data.split(' ')[0]) 

            try:
                decoded_data = db.decode_message(frame_id, data)
                print(f"Timestamp: {timestamp}, Interface: {can_interface}, Frame ID: {frame_id}, Data: {decoded_data}")
            except Exception as e:
                print(f"{e}")

process_log_file('C:/Users/U/Documents/4.Semester/Masterarbeit/concept_implementation/data/CAN_data/20231122_harvester_can_pcan\can_2023-11-22T130939_#000.log')

Timestamp: (1700654935.485798), Interface: can0, Frame ID: 385, Data: {'RPM_Diesel': 0, 'Cool_Temp': 28, 'Oil_Temp': 25, 'Amb_Temp': 0}
Timestamp: (1700654935.485798), Interface: can0, Frame ID: 386, Data: {'FuelPressure': 0.0, 'ChargedAir_Press': 1.0, 'Bat_Volt': 254, 'Oil_Press': 0.0, 'Engine_Load': 0.0}
Timestamp: (1700654935.485798), Interface: can0, Frame ID: 387, Data: {'RPM_DriveMot': 0, 'Speed': 0.0, 'Steering_Angle': 2.0, 'Drive_Dir': 0}
Timestamp: (1700654935.485799), Interface: can0, Frame ID: 388, Data: {'Supply_Press': 0, 'RetFilt_Press': 0.0, 'DrFor_Press': 0, 'DrBack_Press': 0}
Timestamp: (1700654935.485799), Interface: can0, Frame ID: 389, Data: {'Cab_Angle': 1.9000000000000001, 'FrameLR_Angle': 0.5, 'FrameFB_Angle': 0.0}
Timestamp: (1700654935.487012), Interface: can0, Frame ID: 390, Data: {'WorkPump_Press': 0.0, 'WorkPump_Q': 120.0, 'HeadPump_Angle': 0, 'CraneSwivel_Angle': 0.2}
Timestamp: (1700654935.495795), Interface: can0, Frame ID: 385, Data: {'RPM_Diesel': 0, 'C

In [5]:
import cantools
import os

db = cantools.database.load_file('C:/Users/U/Documents/4.Semester/Masterarbeit/concept_implementation/data/CAN_data/MassCAN_DBC_20231108_SmartForestry.dbc')

def process_log_file(log_file_path):
    with open(log_file_path, 'r') as file:
        for line in file:
            try:
                parts = line.strip().split(' ')
                timestamp = parts[0]
                can_interface = parts[1]
                frame_id, data = parts[2].split('#')
                frame_id = int(frame_id, 16) 
                data = bytes.fromhex(data.split(' ')[0]) 

                try:
                    decoded_data = db.decode_message(frame_id, data)
                    data_entry = {
                        "file": os.path.basename(log_file_path),
                        "timestamp": timestamp,
                        "interface": can_interface,
                        "frame_id": frame_id,
                        "decoded_data": decoded_data
                    }
                    all_decoded_data.append(data_entry)
                except Exception as e:
                    print(f"Error decoding frame: {e}")
            except IndexError:
                print(log_file_path)

log_files_directory = 'C:/Users/U/Documents/4.Semester/Masterarbeit/concept_implementation/data/CAN_data/20231123_harvester_can_pcan'
all_decoded_data = []

for file_name in os.listdir(log_files_directory):
    if file_name.endswith('.log'):
        process_log_file(os.path.join(log_files_directory, file_name))

C:/Users/U/Documents/4.Semester/Masterarbeit/concept_implementation/data/CAN_data/20231123_harvester_can_pcan\candump-2023-11-23_163140.log


In [24]:
import pandas as pd

aggregated_data = {}
i = 0 

for entry in all_decoded_data:
    rounded_timestamp = "{:.2f}".format(float(entry['timestamp'].strip('()')))
    key = (entry['file'], rounded_timestamp)
    if key not in aggregated_data:
        aggregated_data[key] = entry['decoded_data']
    else:
        for param, value in entry['decoded_data'].items():
            if param not in aggregated_data[key]:
                aggregated_data[key][param] = value
            else:
                i = i + 1
                aggregated_data[key][param] = value  

final_data = []
for (file, timestamp), decoded_data in aggregated_data.items():
    final_data.append({
        'file': file,
        'timestamp': timestamp,
        'decoded_data': decoded_data
    })

transformed_data = []

for entry in final_data:
    new_entry = {key: value for key, value in entry.items() if key != 'decoded_data'}
    new_entry.update(entry['decoded_data'])
    transformed_data.append(new_entry)


In [30]:
import csv
csv_file_path = 'transformed_data.csv'

all_keys = set()
for data in transformed_data:
    all_keys.update(data.keys())

sorted_keys = sorted(all_keys)

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=sorted_keys)
    writer.writeheader()
    for data in transformed_data:
        row = {key: data.get(key) for key in sorted_keys}
        writer.writerow(row)

Data saved to transformed_data.csv


In [31]:
import csv

transformed_data_truncated = transformed_data[0:100]
csv_file_path = 'transformed_data_truncated.csv'

all_keys = set()
for data in transformed_data_truncated:
    all_keys.update(data.keys())

sorted_keys = sorted(all_keys)

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=sorted_keys)

    writer.writeheader()

    for data in transformed_data_truncated:
        row = {key: data.get(key) for key in sorted_keys}
        writer.writerow(row)

print(f"Data saved to {csv_file_path}")

Data saved to transformed_data_truncated.csv


In [32]:
import sqlite3
db_file_path = 'can_data.db'
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

all_keys = set()
for data in transformed_data:
    all_keys.update(data.keys())

create_table_query = f"CREATE TABLE IF NOT EXISTS CANData ({', '.join([key + ' TEXT' for key in all_keys])})"
cursor.execute(create_table_query)

for data in transformed_data:
    columns = ', '.join(data.keys())
    placeholders = ':' + ', :'.join(data.keys())
    insert_query = f"INSERT INTO CANData ({columns}) VALUES ({placeholders})"
    cursor.execute(insert_query, data)
conn.commit()
conn.close()


Data saved to the database can_data.db
